__*NOTE*__: the official documentation tells you to set the Class' path inside the `default.config` file in the `montepython` folder. I have tried to use the Class folder inside the Linux subsystem, specifically inside the `code` subfolder, but the run raised some issues concerning the filepath of some Class' modules. Now the filepath inside `default.config` is simply set to `/home/errez`.

__*NOTE*__: here, for each run, I specified `python2.7` because this is the supported version for MontePython 2.2.2. For MontePython 3 you can instead use any Python 3 version.

### Input Parameter File
An example of input parameter file is provided with the download package (inside the `code/montepython` folder), under the name `example.param`. Here, we split the file into multiple sub-sections, providing detailed explanation on each of them. Input files are organised as follows:\
\
*__Experiments to test__*: list there all the experiments you want to take into account, separated with commas. Their name should coincide with the name of one of the several sub-directories likelihoods in the `montepython/likelihoods/` directory. Likelihoods will be explained in the dedicated module.\
Here you can also add `data.over_sampling = [1, x]` that means that, for every likelihood evaluation with a fixed configuration of cosmological parameters, you get $x$ evaluations with different combinations of nuisance parameters: this will improve convergence speed.

    #------Experiments to test (separated with commas)-----
    # To see all the available ones, please look in the
    # montepython/likelihoods folder. Every folder there is a valid likelihood to
    # test.
    data.experiments=['fake_planck_bluebook']
    #data.experiments=['wmap','spt']
    
    # Settings for the over sampling
    #data.over_sampling = [1,4]

*__Cosmological Parameter list__*: you can list all the cosmo and nuisance parameter that you want to vary in the Markov chains. For each of them you must give an array with six elements, in this order:
- __mean value__ (your guess for the best fitting value, from which the first jump will start)
- __minimum value__ (set to $-1$ or None for unbounded prior edge),
- __maximum value__ (set to $-1$ or None for unbounded prior edge),
- __sigma__ (your guess for the standard deviation of the posterior of this parameter, its square will be used as the variance of the proposal density when there is no covariance matrix including this parameter passed as an input),
- __scale__ (most of the time, it will be $1$, but occasionally you can use a rescaling factor for convenience, for instance {$1.e-9$} if you are dealing with $A_s$ or $0.01$ if you are dealing with $ω_b$),
- __role__ (cosmo for MCMC parameters used by the Boltzmann code, nuisance for MCMC parameters used only by the likelihoods, and derived for parameters not directly varied by the MCMC algorithm, but to be kept in the chains for memory).

These values do __NOT__ represent the *prior* knowledge (to be contained in the prior distribution).\
The min/max values are related to a prior knowledge about the boundary limits of the distribution: for example, if the parameter is positive by definition (e.g. logarithmic) we would set the minimum value to $0$.

    #------ Parameter list -------
    # data.parameters[class name] = [mean, min, max, 1-sigma, scale, role]
    # - if min max irrelevant, put to -1 or None (if you want a boundary of -1, use -1.0)
    # - if fixed, put 1-sigma to 0
    # - if scale irrelevant, put to 1, otherwise to the appropriate factor
    # - role is either 'cosmo', 'nuisance' or 'derived'

    # Cosmological parameters list
    data.parameters['omega_b'] 	    = [2.249,  -1,-1, 0.016, 0.01,'cosmo']
    data.parameters['omega_cdm'] 	= [0.1120, -1,-1, 0.0016,1,   'cosmo']
    data.parameters['n_s']       	= [0.963,  -1,-1, 0.004, 1,   'cosmo']
    data.parameters['A_s']       	= [2.42,   -1,-1, 0.038, 1e-9,'cosmo']
    data.parameters['h']         	= [0.703,  -1,-1, 0.0065,1,   'cosmo']
    data.parameters['tau_reio']    	= [0.085,  -1,-1, 0.0044,1,   'cosmo']

*__Nuisance and Derived Parameter list__*: they are passed using the same call as the previous parameters, except the name does not have to be a class name. For nuisance parameters, their names must coincide with the ones defined inside the likelihood.\
Derived parameters do NOT affect the run and are computed from other parameters: if you forgot to add one of them during the run, you can easily get them in post-processing, for example using GetDist's `.addDerived()` method. MontePython does NOT read any of the values of the derived parameters' arrays.

    # Nuisance parameter list, same call, except the name does not have to be a class name
    # For wmap:
    #data.parameters['A_SZ']         = [1,    0, 2,   1, 1,  'nuisance']
    # For SPT
    #data.parameters['SPT_SZ']       = [5.6,  0,-1, 2.5, 1,  'nuisance']
    #data.parameters['SPT_PS']       = [20.4,-1,-1, 2.7, 1, 'nuisance']
    #data.parameters['SPT_CL']       = [5.3, -1,-1, 2.3, 1, 'nuisance']

    # Derived parameter list
    data.parameters['z_reio']       = [0,       -1, -1, 0,1,  'derived']
    data.parameters['Omega_Lambda'] = [0,       -1, -1, 0,1,  'derived']

    # Over_sampling settings
    data.over_sampling              = [1, 4]

__*Fixed Parameters*__: here you can pass to the Boltzmann solver (Class) any parameter that you want to fix to a non-default value. The names and values should be the same as in a Class input file:

    data.cosmo_arguments['cosmo_name']  = value

    # Example
    data.cosmo_arguments['Y_He']        = 0.25
    # or
    data.cosmo_arguments['Y_He']        = 'BBN'
    data.cosmo_arguments['sBBN file']   = data.path['cosmo']+'/bbn/sBBN.dat'

__*MCMC Parameters*__: these values adjust the MCMC algorithm parameters. The *number of steps taken* refers to the number of iterations or transitions the Markov chain undergoes during the sampling process. The Markov chain is used to explore the state space of a target probability distribution, and each step corresponds to a change in the current state of the chain.\
Each step comprehends the *acceptance/rejection* process, at the core of the algorithm itself.\
\
Typically, you will need a rather low number here, e.g. `data.write_step = 5` or `10`. The reason for not setting this parameter to one is just to save a bit of time in writing on the disk.

In general, you will want to specify the number of steps in the command line, with the option `-N`. This will overwrite the value passed in the input parameter file. The value by default in the parameter file, `data.N = 10`, is intentionally low, simply to prevent doing any mistake while testing the program on a cluster.

    #------ Mcmc parameters ----
    # Number of steps taken, by default (overwritten by the -N command)
    data.N=10
    # Number of accepted steps before writing to file the chain. Larger means less
    # access to disc, but this is not so much time consuming.
    data.write_step=5

#### New Parameters
All elements you input with a `cosmo`, `derived` or `cosmo_arguments` role will be interpreted by the cosmological code (only Class so far). They are not coded anywhere inside Monte Python. Monte Python takes parameter names, assigns values, and passes all of these to Class as if they were written in a Class input file. The advantages of this scheme are obvious. If you need to fix or vary whatever parameter known by Class, you don’t need to edit Monte Python, you only need to write these parameters in the input parameter file. Also, Class is able to interpret input parameters from a Class input file with a layer of simple logic, allowing to specify different parameter combinations. Parameters passed from the parameter file of Monte Python go through the same layer of logic.

If a `cosmo`, `derived` or `cosmo_arguments` parameter is not understood by the Boltzmann code, Monte Python will stop and return an explicit error message. A similar error will occur if one of the likelihoods requires a nuisance parameter that is not passed in the list.

You may wish occasionally to use in the MCMC runs a new parameter that is not a Class parameter, but can be mapped to one or several Class parameters (e.g. you may wish to use in your chains $\log{(10^{10}A_s)}$ instead of $A_s$). There is a function, in the module data, that you can edit to define such mappings: it is called `update_cosmo_arguments`. Before calling CLASS, this function will simply substitute in the list of arguments your customized parameters by some Class parameters. Several exeamples of such mappings are already implemented, allowing you for instance to use 'Omega_Lambda', 'ln10^{10}A_s' or 'exp_m_2_tau_As' in your chains. Looking at these examples, the user can easily write new ones even without knowing python.

### Output directory
You are assumed to use the code in the following way: for every set of experiments and parameters you want to test, including different priors, some parameters fixed, etc ... you should use __one output folder__. This way, the folder will keep track of the exact calling of the code, allowing you to reproduce the data at later times, or to complete the existing chains. All important data are stored in your `folder/log.param` file.

Incidentaly, if you are starting the program in an existing folder, already containing a `log.param` file, then you do not even have to specify a parameter file: the code will use it automatically. This will avoid mixing things up. If you are using one anyway, the code will warn you that it did not read it: it will always only use the `log.param` file.

In the folder `montepython`, you can create a folder `chains` where you will organize your runs e.g. in the following way:

    # One folder for each chain, associated to a particular set of experiments and model
    montepython/chains/set_of_experiments1/model1
    montepython/chains/set_of_experiments1/model2
    ...
    montepython/chains/set_of_experiments2/model1
    montepython/chains/set_of_experiments2/model2
    ...

The minimum amount of command lines for running Monte Python is an input file, an output directory and a configuration file: if you have already edited `defaut.conf` or copied it to your own `my-machine.conf`, you may already try a mini-run with the command:

    montepython]$ python2.7 montepython/MontePython.py -conf my-machine.conf -p example.param -o test

### Analyzing chains and plotting
Once you have accumulated a few chains, you can analyse the run to get convergence estimates, best-fit values, minimum credible intervals, a covariance matrix and some plots of the marginalised posterior probability. You can run again Monte Python with the `info` prefix followed by the name of a directory or of several chains, e.g. `info chains/myrun/` or `info chains/myrun/2012-10-26* chains/myrun/2012-10-27*`. There is no need to pass an input file with parameter names since they have all been stored in the log.param.

Information on the acceptance rate and minimum of $-\log{\mathcal{L}}=\chi^2_{eff}/2$ is written in `chains/myrun/myrun.log`. Information on the convergence (Gelman-Rubin test for each chain parameter), on the best fit, mean and minimum credible interval for each parameter at the $68.26%$, $95.4%$, $99.7%$ level are written in horizontal presentation in `chains/myrun/myrun.h_info`, and in vertical presentation in `chains/myrun/myrun.v_info` (without $99.7%$ in the vertical one). A latex file to produce a table with parameter names, means and $68%$ errors in written in `chains/myrun/myrun.tex`.

The covariance matrix of the run is written in `chains/myrun/myrun.covmat`. It can be used as an input for the proposal density in a future run. The first line, containing the parameter name, will be read when the covariance matrix will be passed in input. This means that the list of parameters in the input covariance matrix and in the run don’t need to coincide: the code will automatically eliminate, add and reorder parameters (see `mcmc.get_covariance_matrix()`). Note that the rescaling factors passed in the input file are used internally during the run and also in the presentation of results in the `.h_info`, `.v_info`, `.tex` files, but not in the covariance matrix file, which refers to the true parameters.

The 1D posteriors and 2D posterior contours are plotted in `chains/myrun/plots/myrun_1D.pdf` and `chains/myrun/plots/myrun_triangle.pdf`. You will find in the Parser module documentation a list of commands to customize the plots.

When the chains are not very converged and the posterior probability has local maxima, the code will fail to compute minimum credible intervals and say it in a warning. The two solutions are either to re-run and increase the number of samples, or maybe just to decrease the number of bins with the `--bins` option.

### Global running strategy
In the current version of Monte Python, we deliberately choose not to use MPI communication between instances of the code. Indeed the use of MPI usually makes the installation step more complicated, and the gain is, in our opinion, not worth it. Several chains are launched as individual serial runs (if each instance of Monte Python is launched on several cores, Class and the WMAP likelihood will parallelize since they use OpenMP). They can be run with the same command since chain names are created automatically with different numbers for each chain: the chain names are in the form `yyyy-mm-dd_N__i.txt` where `yyyy` is the year, `mm` the month, `dd` the day, `N` the requested number of steps and `i` the smallest available integer at the time of starting a new run.

However the absence of communication between chains implies that the proposal density cannot be updated automatically during the initial stage of a run. Hence the usual strategy consists in launching a first run with a poor (or no) covariance matrix, and a low acceptance rate; then to analyze this run and produce a better covariance matrix; and then to launch a new run with high acceptance rate, leading to nice plots. Remember that in order to respect strictly markovianity and the Metropolis Hastings algorithm, one should not mix up chains produced with different covariance matrices: this is easy if one takes advantage of the info syntax, for example `info chains/myrun/2012-10-26_10000*`. However mixing runs that started from very similar covariance matrices is harmless.

It is also possible to run on several desktops instead of a single cluster. Each desktop should have a copy of the output folder and with the same `log.param` file, and after running the chains can be grouped on a single machine and analyse. In this case, take care of avoiding that chains are produced with the same name (easy to ensure with either the `-N` or `--chain-number` options). This is a good occasion to keep the desktops of your department finally busy.

### Work Session
First of all, create a few folders in order to keep the `montepython` folder tidy in the future:
- `chains` for storing all the chains,
- `chains/run_name` for storing the chains related to a particular experiment (e.g. `run_name=planck`),
- `input` for storing all the parameter input files,
- `scripts` for storing all the scripts for running the code in batch mode.

Then we need to pass MontePython the parameter input file - described earlier: to do so, copy the `<name>.param` file inside the input folder and launch a short chain with:

    # Very short chain
    $ python2.7 montepython/MontePython.py run -p input/<name>.param -o chains/run_name/<name> -N 5

You can see on the screen the evolution of the initialization of the code. At the end you can check that you have a chain and a `log.param` written inside the `chains/run_name/<name>/log.param` directory. You can immediately repeat the experience with the same command. The second chain is automatically created with number 2 instead of 1. You can also run again without the input file:

    $ python2.7 montepython/MontePython.py run -o chains/run_name/<name> -N 5

In some cases, initally, you may not have a covariance matrix to pass in input. Otherwise, if you have previously stored the covariance matrix inside a `covmat/` folder, naming the file `mycovmat.covmat`, you can pass it to MontePython with:

    $ python2.7 montepython/MontePython.py run -p input/<name>.param -o chains/run_name/<name> -c covmat/mycovmat.covmat -N 5

The covariance matrix is used as *proposal distribution* to initialize the chain: therefore, giving a more accurate covariance matrix will produce as well a more accurate and converged chain.

You can also add a `.bestfit` file using the command `-b` in order to give more precise starting points for the chain.

If you want to use a certain likelihood in a future run (let's say the Planck likelihood), you should not forget to add in the script (before calling Monte Python) the line:

    source /path/to/my/plc/bin/clik_profile.sh

You can then launch a chain by submitting the script, with e.g. `qsub scripts/lcdm.sh`. You can launch many chains in one command with:

    $ for i in {1..10}; do qsub scripts/lcdm.sh;done

However the cluster may create the chains too fast, therefore leading to name conflicts. You can solve this issue by running a parallelization process, using the `mpi` command: to launch a job with `mpi`, the syntax is exactly the same than without, except that you will start the whole command with, depending on your installation, `mpirun` or `mpiexec`.\
The next line will simply launch 4 chains, each using the environment variable `$OMP_NUM_THREADS` for the number of cores to compute Class.

    $ mpirun -np 4 python2.7 montepython/MontePython.py run -o chains/...

You can analyse the chains using:

    $ python2.7 montepython/MontePython.py info chains/run_name/<name>

If you had been running without a covariance matrix, the results would probably be bad, with a very low acceptance rate and few points. This process, though, should have created a covariance matrix inside `chains/run_name/<name>/<name>.covmat`. You can copy it in order to keep track of it even after analysing future runs:

    $ cp chains/run_name/<name>/<name>.covmat chains/run_name/<name>/<name>_run1.covmat

You can now add to the script, in the line starting with `montepython/MontePyhton.py`, the option:

    $ ... -c chains/run_name/<name>/<name>_run1.covmat

If you run on the same day as the previous one (the run without the covaraince matrix), it might be smart to change also a bit the number of steps (e.g. from $5000$ to $5001$) in order to immediately identify chains belonging to the same run but with different elements.

When this second run is finished, you can analyse it with e.g.:

    # The last part of the name refers to the date plus the number of steps.
    $ python2.7 montepython/MontePython.py info chains/run_name/<name>/2012-10-27_5001*

#### Final Adjustments
If all $R-1$ numbers are small (typically $<0.05$) and plots look nice, you can consider the session closed. If not, there can be two reasons: the covariance matrix is still bad, or you just did not get enough samples.

You can check the acceptance rate of this last run by looking at the `chains/run_name/<name>/<name>.log` file. If you're in a case with nearly gaussian posterior (i.e. nearly ellipsoidal contours), an acceptance rate $<0.2$ or $>0.3$ can be considered as bad. In other cases, even $0.1$ might be the best that you can expect. If the acceptance rate is bad, you must re-run with an improved covariance matrix in order to converge quicker. You can copy the last covariance matrix to `<name>_run2.covmat` and use this one for the next run. If the acceptance rate is good but the chains are not well converged because they are simply too short, then you should better rerun with the same covariance matrix `<name>_run1.covmat`: in this way, you know that the proposal density is frozen since the second run, and you can safely analyse the second and third runs altogether.

If you do two or three runs in that way, you always loose running time, because each new chain will have a new burn-in phase (i.e. a phase when the log likelihood is very bad and slowly decreasing towards values close to the minimum). If this is a concern, you can avoid it in three ways:
- before launching the new run, set the input mean value of each parameter in the input file to the best-fit value found in the previous run. The runs will then start from the best-fit value plus or minus the size of the first jump drown from the covariance matrix, and avoid burn-in. Since you have changed the input file, you must rerun with a new output directory, e.g. `chain/<name>2`. This is a clean method.
 - You might prefer a less clean but slightly quicker variant: modify the mean values, like in the previous item, but directly in the `log.param` file, and rerun in the same directory without an input file. This will work, but it is advisable not to edit the `log.param` manually, since it is supposed to keep all the information from previous runs.
 - Restart the new chains from the previous chains using the `-r` command line option. The name of previous chains can be written after `-r` manually or through a script.
 - You can also restart from the best-fit found previously, using the `-bf` command line option, specifying the `.bestfit` file to use.

When you are pleased with the final plots and results, you can customize the plot content and labels by writing a short file `plot_files/<name>.plot` passed through the `-extra` command line option, and paste the latex file produced by Monte Python in your paper.

### Priors and Likelihoods
First of all, we must note that adding a prior distribution in Monte Pyhton is equivalent to adding a new likelihood. This means that if you want to add a prior on a particular parameter (let's say, $\Omega_b$ from Planck observations) you should follow these steps:
1. Take a simple likelihood folder as an example, for instance `hst`. Rename it as you prefer, i.e. `cmb_baryon`. This folder contains two files: `__init__.py` and `hst.data`. Rename also the latter one to `cmb_baryon.data` for consistency.

2. Now look at the `hst.data` file:
    ```
    # Values for Hubble Space Telescope
    # Updated to Riess et al. 2016 (arXiv:1604.01424)
    hst.h      = 0.7324
    hst.sigma  = 0.0174
    ```
    Here you can see that adding a prior is as easy as typing two values: the mean and standard deviation of the normal distribution (more complex priors require much more work, check the *Issues* section on the official GitHub page).\
    To continue with the previous example, here we should apply the following changes in the `.data` file: `hst -> cmb_baryon`, `h -> omega_b` and update the mean and standard deviation values.

3. Finally, look at the `__init__.py` file:
    ```
    import os
    from montepython.likelihood_class import Likelihood_prior
    class hst(Likelihood_prior):
    # Initialisation of the class is done within the parent Likelihood_prior. For
    # this case, it does not differ, actually, from the __init__ method in
    # Likelihood class.
    def loglkl(self, cosmo, data):
        h = cosmo.h()
        loglkl = -0.5 * (h - self.h) ** 2 / (self.sigma ** 2)
        return loglkl
    ```
    Now you can see that we only need to define a class, that inherits from the `Likelihood_prior` class, which contains a `loglkl()` (log-likelihood) function with 3 arguments: `self`, `cosmo`, used to call CLASS parameters, and `data`, specifies the name of the data file. Just rename the class as `cmb_baryon`, and all the `h`'s to `omega_b`.

4. Now you can try to run the following code to produce a *prior distribution chain*. You should notice that the chains are roughly gaussianly distributed around the mean:
    ```
    $ mpi -np 4 python montepython/MontePython.py -p input/omega_b_prior.param -o chains/omega_b_prior -N 10000
    
    ```
    where the file named `input/omega_b_prior.param` is written as follows, connecting the cosmological parameter $\omega_b \equiv \Omega_b h^2$ to its prior definition:
    ```
    data.experiments=['cmb_baryon']
    data.parameters['omega_b'] = [2, 0, None, 0.02, 1e-2, 'cosmo']
    data.N=10
    data.write_step=5